In [1]:
import numpy as np 
import pandas as pd 
import os

# COVID-19 knowledge graph

Relations between chemicals, genes and disease offer insights into the mechanisms behind higher order biochemical phenomena, such as drug-drug interactions, drug response and gene-disease associations. This work focused on extracting relations between chemicals, genes and disease from research articles to build a biomedical knowledge graph related to COVID-19. We used entities collected by BioSNAP to identify instances of chemical, gene and disease names in abstracts.

To understand what relationship types are possible and map unstructured natural language descriptions onto these structured classes, we used labeled sentences in Percha B 2018 to classify relations bewteen entities.

In [2]:
!pip install pyecharts

     |████████████████████████████████| 128 kB 4.9 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pyecharts import options as opts
from pyecharts.charts import Graph
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from keras.preprocessing import sequence
from transformers import BertTokenizer, BertForSequenceClassification

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

Using TensorFlow backend.


**Load marked sentences**
These sentences containing more than two biomedical entities were extracted from abstracts and stored in a csv file. We combined the pairwise entities in each sentence to build relations.

In [4]:
marked_sentence_df = pd.read_csv('/kaggle/input/covid19-knowledge-graphs-relation-files/marked_sentence/marked_sentence.csv')
marked_sentence_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence
6660,Diabetes Mellitus,cough,Disease,Disease,start_entity correlated with an increased risk...
751,Melanoma,"Paraneoplastic Syndromes, Ocular",Disease,Disease,As far as the retina is concerned end_entity a...
16684,Bleomycin,Fibrosis,Chemical,Disease,"Lemessurier, Josepha Moore, Jeoung-Eun Park, A..."
29058,GCNT2,COVID-19,Gene,Disease,The type start_entity transmembrane serine pro...
16065,CYP2R1,"Dermatitis, Contact",Gene,Disease,Pieters A245 How to Define Chronic Cough: Base...


In [5]:
def Build_graph(df,relation=False,repulsion=40,title='COVID-19 knowledge graph',labelShow=False):
    entity_type_dic = dict(df.drop_duplicates(['start_entity']).set_index(['start_entity'])['start_entity_type'])
    entity_type_dic.update(dict(df.drop_duplicates(['end_entity']).set_index(['end_entity'])['end_entity_type']))
    color = {'Disease':'#FF7F50','Gene':'#48D1CC','Chemical':'#B3EE3A'}
    cate =  {'Disease':0,'Gene':1,'Chemical':2}
    categories = [{'name':'Disease','itemStyle': {'normal': {'color': color['Disease']}}},{'name':'Gene','itemStyle': {'normal': {'color': color['Gene']}}},{'name':'Chemical','itemStyle': {'normal': {'color': color['Chemical']}}}]
    nodes = []
    for entity in list(set(df['start_entity'])|set(df['end_entity'])):
        nodes.append({'name': entity, 'symbolSize': max(10,np.log1p(df.loc[(df['start_entity']==entity)|(df['end_entity']==entity)].shape[0])*10//1),
                     'category':cate[entity_type_dic[entity]]})
    links = []
    for i in df.index:
        if not relation:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'],'value':df.loc[i,'marked_sentence']})
        else:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'], 'value':df.loc[i,'theme']})
    g = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render_notebook()
        )
    g1 = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render(path=title+'.html')
        )
    return g


In [6]:
g = Build_graph(marked_sentence_df.sample(800),title='subsample of topology graph')
g

# Classify relations
Percha B 2018 revealed 10 broad themes for chemical-gene relations, 7 for chemical-disease, 10 for gene-disease and 9 for gene–gene in Medline abstracts. We used these labeled sentences to train a model to classify sentences in this work. As we cannot confirm order of entities, we predicted two possibilities(init_pred and reverse_pred) and retrieved the higher one. The details can be seen here

In [7]:
class Args:
    task_type = 'chemical-disease'
    max_seq_len = 64
    bs = 64

class Conf:
    # some information can be found in:
    # Percha B, Altman R B. A global network of biomedical relationships derived from text[J]. Bioinformatics, 2018, 34(15): 2614-2624.
    relation_type = {'chemical-disease':['T', 'C', 'Sa', 'Pr', 'Pa', 'J'],
                     'disease-chemical':['Mp'],
                     'chemical-gene':['A+', 'A-', 'B', 'E+', 'E-', 'E', 'N'],
                     'gene-chemical':['O', 'K', 'Z'],
                     'gene-disease':['U', 'Ud', 'D', 'J', 'Te', 'Y', 'G'],
                     'disease-gene':['Md', 'X', 'L'],
                     'gene-gene':['B', 'W', 'V+', 'E+', 'E', 'I', 'H', 'Rg', 'Q'],
                     }

args = Args()
conf = Conf()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
# load pretrained Bert model
def Bert_model(taskType,bertPath):
    label_df = pd.read_csv('/kaggle/input/covid19-knowledge-graph/%s_label.csv'%taskType)
    tokenizer = BertTokenizer.from_pretrained(bertPath,do_lower_case=False)
    model = BertForSequenceClassification.from_pretrained(bertPath, num_labels=label_df['label'].nunique())
    return label_df,tokenizer,model

# bulid data loader
def Data_loader(x,y=None,bs=128,shuffle=False,numWorkers=0):
    if y is not None:
        data = TensorDataset(x,y)
    else:
        data = TensorDataset(x)
    data_loader = DataLoader(dataset=data,batch_size=bs,shuffle=shuffle,num_workers=numWorkers)
    return data_loader

def Prepare_predict_data(tokenizer,bs):
    marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']
    # convert tokens to ids
    ids = marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    # padding ids
    ids = sequence.pad_sequences(ids,args.max_seq_len, truncating='post', padding='post')
    # we cannot confirm order of entities, so predict two possibilities
    reverse_marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']\
                                              .apply(lambda x:x.replace('start_entity','init_start_entity').replace('end_entity','start_entity').replace('init_start_entity','end_entity'))
    reverse_ids = reverse_marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    reverse_ids = sequence.pad_sequences(reverse_ids,args.max_seq_len, truncating='post', padding='post')
    predict_data_loader = Data_loader(torch.LongTensor(ids),torch.LongTensor(reverse_ids),bs=bs)
    return marked_sentences.values,predict_data_loader

def Predict():
    reverse_task_type = args.task_type.split('-')[1] + '-' + args.task_type.split('-')[0]
    def Filter(x):
        if x['init_pred'] in conf.relation_type[args.task_type]:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred is a correct relation but reverse_pred not
                return 'init_pred'
            else:
                # init_pred and reverse_pred both are correct relations
                if x['init_pred_prob'] >= x['reverse_pred_prob']:
                    # init_pred_prob greater than or equal to reverse_pred_prob
                    return 'init_pred'
                else:
                    return 'reverse_pred'
        else:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred and reverse_pred both are uncorrect relations
                return 'uncorrect'
            else:
                # reverse_pred is a correct relation but init_pred not
                return 'reverse_pred'
    label_df,tokenizer,model = Bert_model(args.task_type,'/kaggle/input/covid19-knowledge-graph/%s/'%args.task_type)
    marked_sentences,predict_data_loader = Prepare_predict_data(tokenizer,args.bs)
    model = model.to(device)
    preds = []
    preds_prob = []
    reverse_preds = []
    reverse_preds_prob = []
    for data in tqdm(predict_data_loader):
        ids,reverse_ids = [t.to(device) for t in data]
        outputs = model(input_ids=ids)
        logits = outputs[0]
        pred_prob, pred = torch.max(F.softmax(logits.data,1), 1)
        preds.extend(list(pred.cpu().detach().numpy()))
        preds_prob.extend(list(pred_prob.cpu().detach().numpy()))
        reverse_outputs = model(input_ids=reverse_ids)
        reverse_logits = reverse_outputs[0]
        reverse_pred_prob, reverse_pred = torch.max(F.softmax(reverse_logits.data,1), 1)
        reverse_preds.extend(list(reverse_pred.cpu().detach().numpy()))
        reverse_preds_prob.extend(list(reverse_pred_prob.cpu().detach().numpy()))

    pred_df = pd.DataFrame({'marked_sentence':marked_sentences,'init_pred':preds,'init_pred_prob':preds_prob,'reverse_pred':reverse_preds,'reverse_pred_prob':reverse_preds_prob})
    # map label(0, 1, 2...) to raw label(T, C, Sa...) 
    pred_df['init_pred'] = pred_df['init_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    pred_df['reverse_pred'] = pred_df['reverse_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    # judge the order of a pair of entities
    pred_df['filter'] = pred_df.apply(lambda x:Filter(x), axis=1)
    pred_df['pred'] = pred_df['init_pred']
    pred_df['pred_prob'] = pred_df['init_pred_prob']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred_prob'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred_prob']
    pred_df = pred_df.loc[pred_df['filter']!='uncorrect']
    pred_df = marked_sentence_df.merge(pred_df,how='inner',on='marked_sentence')
    pred_df['init_start_entity'] = pred_df['start_entity']
    pred_df['init_start_entity_type'] = pred_df['start_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity_type']
    pred_df.drop(['init_start_entity','init_start_entity_type'],axis=1,inplace=True)
    torch.cuda.empty_cache()
    return label_df,pred_df
# chemical-disease relation prediction
args.task_type = 'chemical-disease'
import pickle

try:
    c_d_label_df,c_d_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/chem_disease.pickle", "rb"))
except (OSError, IOError) as e:
    c_d_label_df,c_d_pred_df = Predict()
    pickle.dump((c_d_label_df,c_d_pred_df), open("chem_disease.pickle", "wb"))

In [9]:
# chemical-disease relation theme
c_d_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
c_d_label_df

,pred,label,theme
0,C,0,Inhibits cell growth (esp. cancers)
1,J,1,Role in pathogenesis
2,Mp,2,Biomarkers (progression)
3,Pa,3,"Alleviates, reduces"
4,Pr,4,"Prevents, suppresses"
5,Sa,5,Side effect/adverse event
6,T,6,Treatment/therapy (incl. investigatory)


In [10]:
# chemical-disease classification results
c_d_pred_df=c_d_pred_df.merge(c_d_label_df, on='pred')
c_d_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob,label,theme
375,ventilation,COVID-19,Chemical,Disease,The treatment experience in severe cases of 20...,Pr,0.913143,T,0.991021,init_pred,Pr,0.913143,4,"Prevents, suppresses"
89,Ganciclovir,COVID-19,Chemical,Disease,start_entity can be considered for patients wi...,T,0.999927,T,0.999898,init_pred,T,0.999927,6,Treatment/therapy (incl. investigatory)
556,Allopurinol,Asthma,Chemical,Disease,"Ehlayel, Abdul Bari Bener A607 Strong Associat...",C,0.947384,Pr,0.701428,init_pred,C,0.947384,0,Inhibits cell growth (esp. cancers)
500,Ginseng,Inflammation,Chemical,Disease,Pieters A245 How to Define Chronic Cough: Base...,C,0.527133,C,0.527133,init_pred,C,0.527133,0,Inhibits cell growth (esp. cancers)
518,Pranlukast,Anaphylaxis,Chemical,Disease,Pieters A245 How to Define Chronic Cough: Base...,C,0.527133,C,0.527133,init_pred,C,0.527133,0,Inhibits cell growth (esp. cancers)


In [11]:
# chemical-gene relation prediction
args.task_type = 'chemical-gene'
try:
    c_g_label_df,c_g_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/chem_gene.pickle", "rb"))
except (OSError, IOError) as e:
    c_g_label_df,c_g_pred_df = Predict()
    pickle.dump((c_g_label_df,c_g_pred_df), open("chem_gene.pickle", "wb"))

In [12]:
# chemical-gene relation theme
c_g_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
c_g_label_df

,pred,label,theme
0,A+,0,"Agonism, activation"
1,A-,1,"Antagonism, blocking"
2,B,2,"Binding, ligand (esp. receptors)"
3,E,3,Affects expression/production (neutral)
4,E-,4,Decreases expression/production
5,K,5,"Metabolism, pharmacokinetics"
6,N,6,Inhibits
7,O,7,"Transport, channels"
8,Z,8,Enzyme activity


In [13]:
# chemical-gene classification results
c_g_pred_df=c_g_pred_df.merge(c_g_label_df, on='pred')
c_g_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob,label,theme
389,RAGE,Heparin,Gene,Chemical,Background: start_entity and heparin-related s...,A+,0.469736,O,0.990363,reverse_pred,O,0.990363,7,"Transport, channels"
326,Doxycycline,VDR,Chemical,Gene,"Leung, Ka Hou Chu A67 The relationship between...",B,0.729219,B,0.729219,init_pred,B,0.729219,2,"Binding, ligand (esp. receptors)"
211,Sevoflurane,DLG4,Chemical,Gene,"start_entity, a commonly used anesthetic in ch...",E,0.994707,E,0.882591,init_pred,E,0.994707,3,Affects expression/production (neutral)
337,Toluene,POSTN,Chemical,Gene,"Leung, Ka Hou Chu A67 The relationship between...",B,0.729219,B,0.729219,init_pred,B,0.729219,2,"Binding, ligand (esp. receptors)"
72,Metformin,MAPK8,Chemical,Gene,CONCLUSION: start_entity treatment suppresses ...,N,0.998267,E,0.948754,init_pred,N,0.998267,6,Inhibits


In [14]:
# gene-disease relation prediction
args.task_type = 'gene-disease'
try:
    g_d_label_df,g_d_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/gene_disease.pickle", "rb"))
except (OSError, IOError) as e:
    g_d_label_df,g_d_pred_df = Predict()
    pickle.dump((g_d_label_df,g_d_pred_df), open("gene_disease.pickle", "wb"))

In [15]:
# gene-disease relation theme
g_d_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
g_d_label_df

,pred,label,theme
0,D,0,Drug targets
1,G,1,Promotes progression
2,J,2,Role in pathogenesis
3,L,3,Improper regulation linked to disease
4,Md,4,Biomarkers (diagnostic)
5,Te,5,Possible therapeutic effect
6,U,6,Causal mutations
7,Ud,7,Mutations affect disease course
8,X,8,Overexpression in disease
9,Y,9,Polymorphisms alter risk


In [16]:
# gene-disease classification results
g_d_pred_df=g_d_pred_df.merge(g_d_label_df, on='pred')
g_d_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob,label,theme
6534,Retinoblastoma,IL1RAP,Disease,Gene,end_entity was identified as the direct target...,Te,0.531874,L,0.967089,reverse_pred,L,0.967089,3,Improper regulation linked to disease
5201,TREM1,Adrenoleukodystrophy,Gene,Disease,start_entity inhibition represents a promising...,U,0.996784,U,0.950953,init_pred,U,0.996784,6,Causal mutations
2444,HAI,Equine Infectious Anemia,Gene,Disease,Twenty-three antibody rises to OC43 were detec...,J,0.912696,J,0.878945,init_pred,J,0.912696,2,Role in pathogenesis
5966,COVID-19,GALT,Disease,Gene,To address this question and to study interact...,J,0.813042,L,0.909468,reverse_pred,L,0.909468,3,Improper regulation linked to disease
3890,PAEP,COVID-19,Gene,Disease,The rate of MERS-CoV infection was compared be...,Te,0.784639,Te,0.998869,init_pred,Te,0.784639,5,Possible therapeutic effect


In [17]:
# gene-gene relation prediction
args.task_type = 'gene-gene'
try:
    g_g_label_df,g_g_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/gene_gene.pickle", "rb"))
except (OSError, IOError) as e:
    g_g_label_df,g_g_pred_df = Predict()
    pickle.dump((g_g_label_df,g_g_pred_df), open("gene_gene.pickle", "wb"))

In [18]:
# gene-gene relation theme
g_g_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
g_g_label_df

,pred,label,theme
0,B,0,"Binding, ligand (esp. receptors)"
1,E,1,Affects expression/production (neutral)
2,E+,2,Increases expression/production
3,H,3,Same protein or complex
4,I,4,Signaling pathway
5,Q,5,Production by cell population
6,Rg,6,Regulation
7,V+,7,"Activates, stimulates"
8,W,8,Enhances response


In [19]:
# gene-gene classification results
g_g_pred_df=g_g_pred_df.merge(g_g_label_df, on='pred')
g_g_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob,label,theme
12636,E2,E1,Gene,Gene,"A set of three-step cascade enzymes, an start_...",Rg,0.891795,Rg,0.908306,reverse_pred,Rg,0.908306,6,Regulation
14213,MAVS,IFNA1,Gene,Gene,IE1 prevents start_entity- gene expression tri...,Rg,0.992408,Rg,0.997719,reverse_pred,Rg,0.997719,6,Regulation
19398,DBT,L2,Gene,Gene,The extent of coronavirus-induced cell fusion ...,Q,0.996846,Q,0.994313,init_pred,Q,0.996846,5,Production by cell population
17846,STAT5A,STAT1,Gene,Gene,Nave CD4 T cells from young adults upregulate ...,Q,0.982580,Q,0.996001,reverse_pred,Q,0.996001,5,Production by cell population
6767,PAPPA,PGF,Gene,Gene,"RESULTS: 1 Ten percent 1121,125 of the profile...",V+,0.976803,V+,0.976803,init_pred,V+,0.976803,7,"Activates, stimulates"


# Show relation between covid-19 and other entities
pycharts cannot show multiple relations between two entities, so the graph is incomplete. Complete relations can be seen in pred_df.

In [20]:
# chemical-COVID-19 relations
g = Build_graph(c_d_pred_df.loc[(c_d_pred_df['start_entity']=='COVID-19')|(c_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=800,title='chemical-COVID-19 knowledge graph',labelShow=True)
g

In [21]:
# gene-COVID-19 relations
g = Build_graph(g_d_pred_df.loc[(g_d_pred_df['start_entity']=='COVID-19')|(g_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=60,title='gene-COVID-19 knowledge graph',labelShow=True)
g

In [22]:
marked_sentence_df

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence
0,IGKV4-1,TAZ,Gene,Gene,"In addition to rubella there is suggestive, bu..."
1,keyhole limpet hemocyanin,"Lupus Erythematosus, Systemic",Chemical,Disease,"Middle-aged female identical twins, one of who..."
2,COVID-19,Pharyngitis,Disease,Disease,"end_entity, bronchitis, and start_entity repre..."
3,cough,sputum,Disease,Disease,It is characterized by persistent start_entity...
4,B3GAT1,GLS2,Gene,Gene,While the perikarya of all cells contained ele...
...,...,...,...,...,...
40807,Coronavirus Infections,Syndrome,Disease,Disease,In a recent review on the persistence of human...
40808,MERS,Severe Acute Respiratory Syndrome,Disease,Disease,In a recent review on the persistence of human...
40809,MERS,Syndrome,Disease,Disease,In a recent review on the persistence of human...
40810,Severe Acute Respiratory Syndrome,Syndrome,Disease,Disease,In a recent review on the persistence of human...


In [23]:
# gene-COVID-19 relations
g = Build_graph(marked_sentence_df.loc[(marked_sentence_df['start_entity']=='COVID-19')&(marked_sentence_df['end_entity_type']=='Disease')|(marked_sentence_df['start_entity_type']=='Disease')&(marked_sentence_df['end_entity']=='COVID-19')],relation=False,repulsion=60,title='disease-COVID-19 knowledge graph',labelShow=True)
g

# Answering researchers questions

In [24]:
# merge all relation prediction and save results
cols = ['start_entity','end_entity','start_entity_type','end_entity_type','marked_sentence','pred','pred_prob','theme']
relation_df = pd.concat([c_d_pred_df[cols],c_g_pred_df[cols],g_d_pred_df[cols],g_g_pred_df[cols]]).append(marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].isin(['Chemical','Disease'])&(marked_sentence_df['start_entity_type']==marked_sentence_df['end_entity_type']))]).reset_index(drop=True)
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Chemical'),'pred'] = 'CC'
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Disease'),'pred'] = 'DD'
relation_df = relation_df[cols]
relation_df.to_csv('relation.csv',index=False)

In [25]:
labels_df= pd.concat([c_d_label_df,c_g_label_df,g_d_label_df,g_g_label_df])

In [26]:
# subsample of Combined knowledge graph
g = Build_graph(relation_df.sample(1000),relation=True,repulsion=15,title='subsample of COVID-19 knowledge graph',labelShow=False)
g

# Knowledge graph between pair of entities(Covid specific and Non specific also)

In [27]:
ch1='Chemical'
ch2='Disease'
ch3=2

In [28]:
# ch1=input('enter choice 1 : Chemical,Gene,Disease = You selected ')
# ch2=input('enter choice 2 : Chemical,Gene,Disease = You selected ')
# ch3=int(input('enter 2-Covid19 related OR  1- Not specific to Covid  = You selected '))
if ch3==1:
    temp=relation_df.loc[((relation_df['start_entity_type']==ch1)&(relation_df['end_entity_type']==ch2))|((relation_df['start_entity_type']==ch2)&(relation_df['end_entity_type']==ch1))]#
    temp = temp[cols]
    g = Build_graph(temp,relation=True,repulsion=15,title= ch1+'_'+ch2+'_knowledge graph',labelShow=True)
    temp.to_csv(ch1+'_'+ch2+'_knowledge graph.csv')
else:
    temp=relation_df.loc[((relation_df['start_entity_type']==ch1)&(relation_df['end_entity_type']==ch2))|((relation_df['start_entity_type']==ch2)&(relation_df['end_entity_type']==ch1))]#
    temp=temp.loc[(relation_df['start_entity']=='COVID-19')|(relation_df['end_entity']=='COVID-19')]
    temp = temp[cols]
    g = Build_graph(temp,relation=True,repulsion=15,title= 'Covid19 Specific_'+ch1+'_'+ch2+'_knowledge graph',labelShow=True)
    temp.to_csv('Covid19 Specific_'+ch1+'_'+ch2+'_knowledge graph.csv')
g

# Search based on desired relationships

In [29]:
labels_df

,pred,label,theme
0,C,0,Inhibits cell growth (esp. cancers)
1,J,1,Role in pathogenesis
2,Mp,2,Biomarkers (progression)
3,Pa,3,"Alleviates, reduces"
4,Pr,4,"Prevents, suppresses"
5,Sa,5,Side effect/adverse event
6,T,6,Treatment/therapy (incl. investigatory)
0,A+,0,"Agonism, activation"
1,A-,1,"Antagonism, blocking"
2,B,2,"Binding, ligand (esp. receptors)"


In [30]:
import re
ch5='Biomarkers (progression)'
# ch5=input('enter desired relationship')
temp=relation_df.loc[(relation_df['theme']==ch5)]#
if temp is None:
    print('yes')
temp = temp[cols]
ch5=re.sub(r'/', '_', ch5)
g = Build_graph(temp,relation=True,repulsion=15,title=ch5+'_knowledge graph',labelShow=True)
temp.to_csv(ch5+'_knowledge graph.csv')
g